# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../data/flights_samp.csv')

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [3]:
df.columns

Index(['index', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [8]:
# import fresh df from .csv before using this 
def convert_to_test_format (df):
    # flightdate to just date 2020-01-01 00:00:00 no time
    df.fl_date = df.fl_date + ' 00:00:00'
    df.drop(columns=['index', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'actual_elapsed_time', 'air_time', 
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'], inplace = True)
    return df

def convert

In [11]:
df_test = convert_to_test_format (df)

In [12]:
df_test

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2019-01-07 00:00:00 00:00:00,AA,AA,AA,362,AA,N161AA,362,12892,LAX,"Los Angeles, CA",13930,ORD,"Chicago, IL",59,710,N,251.0,1.0,1744.0
1,2019-01-04 00:00:00 00:00:00,UA,UA_CODESHARE,UA,3788,ZW,N437AW,3788,13930,ORD,"Chicago, IL",10469,AZO,"Kalamazoo, MI",2135,2333,N,58.0,1.0,122.0
2,2018-12-29 00:00:00 00:00:00,WN,WN,WN,5741,WN,N7738A,5741,13871,OMA,"Omaha, NE",14107,PHX,"Phoenix, AZ",1640,1835,N,175.0,1.0,1037.0
3,2019-01-01 00:00:00 00:00:00,WN,WN,WN,1641,WN,N423WN,1641,15304,TPA,"Tampa, FL",13495,MSY,"New Orleans, LA",1015,1050,N,95.0,1.0,488.0
4,2019-01-02 00:00:00 00:00:00,UA,UA_CODESHARE,UA,4233,EV,N14558,4233,11618,EWR,"Newark, NJ",11278,DCA,"Washington, DC",1825,1951,N,86.0,1.0,199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2019-01-01 00:00:00 00:00:00,DL,DL,DL,2173,DL,N535US,2173,12892,LAX,"Los Angeles, CA",13487,MSP,"Minneapolis, MN",1100,1640,N,220.0,1.0,1535.0
4996,2019-01-10 00:00:00 00:00:00,G4,G4,G4,1608,G4,253NV,1608,14082,PGD,"Punta Gorda, FL",13230,MDT,"Harrisburg, PA",720,948,N,148.0,1.0,963.0
4997,2019-01-10 00:00:00 00:00:00,DL,DL_CODESHARE,DL,3289,9E,N294PQ,3289,11003,CID,"Cedar Rapids/Iowa City, IA",10397,ATL,"Atlanta, GA",1646,1942,N,116.0,1.0,694.0
4998,2019-01-12 00:00:00 00:00:00,AA,AA,AA,2809,AA,N338RS,2809,12953,LGA,"New York, NY",13303,MIA,"Miami, FL",1929,2249,N,200.0,1.0,1096.0


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.